In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
noise_type = 'gaussian'      # 'gaussian', 'saltpepper', 'speckle'
model_type = 'conv'          # 'fc', 'conv'
epochs = 10
batch = 128
latent_dim = 64

In [ ]:
(train_x, _), (test_x, _) = tf.keras.datasets.mnist.load_data()
train_x = train_x.astype('float32') / 255.
test_x = test_x.astype('float32') / 255.
train_x = np.reshape(train_x, (-1, 28, 28, 1))
test_x = np.reshape(test_x, (-1, 28, 28, 1))


In [ ]:
def add_gaussian_noise(imgs, amount=0.5):
    noise = amount * np.random.normal(loc=0.0, scale=1.0, size=imgs.shape)
    return np.clip(imgs + noise, 0., 1.)